In [ ]:
import pandas as pd
import keras
import numpy as np

from argparse import ArgumentParser

from lib import training as tr
from lib import database as db

In [ ]:
def get_option():
    a = ArgumentParser()
    a.add_argument('-k', '--kind', choices = ['mini', 'loto6', 'loto7'], required = True)
    a.add_argument('-m', '--month', type = int, required = True)
    a.add_argument('-d', '--day', type = int, required = True)
    a = a.parse_args()
    return a

In [ ]:
a = get_option()
kind = a.kind

model = keras.models.load_model('h5/' + kind + '.h5', compile = True)

loto = db.session.query(db.Loto).filter(db.Loto.kind == kind).all()

num_classes = tr.get_num_classes(kind)

x = tr.get_x_data(loto, len(loto))
x_test = np.array([[a.month, a.day] + x])
x_test = x_test.astype('float32')
x_test /= num_classes

y_pred = model.predict(x_test)

In [ ]:
result = {i + 1 : y_pred[0][i] for i in range(num_classes)}
result = sorted(result.items(), key = lambda x:-x[1])
result = [r[0] for r in result]

if kind == 'loto7':
    print(sorted(result[0:7]))

elif kind == 'loto6':
    print(sorted(result[0:6]))

else:
    print(sorted(result[0:5]))
